### Установка пакетов

In [1]:
!pip install pyspark

### Импорт библиотек

In [2]:
import json
import matplotlib.pyplot as plt
import os
from pyspark.ml.stat import ChiSquareTest
from pyspark.ml.linalg import Vectors
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
import urllib.request

### Генерация номера задания

In [3]:
# random.randint(1, 200)

Сгенерирован номер с заданием 99

### Создание каталога data для хранения данных

In [4]:
if not os.path.exists('data'):
  os.makedirs('data')

urllib.request.urlretrieve('https://raw.githubusercontent.com/Losyash/sepbd_i_hw/refs/heads/master/lab_3/data/4.csv', 'data/4.csv')

('data/4.csv', <http.client.HTTPMessage at 0x78d84d543970>)

### Создание экземпляра SparkSession

In [5]:
spark = SparkSession.builder.appName('Homework_3_lab4').getOrCreate()

### Задание 1. Считать файл 4.csv и создать Spark DataFrame

In [6]:
df = spark.read.format('csv').option('header', 'true').load('data/4.csv')
df.show(5)

+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---+------+
|        feature_1|         feature_2|         feature_3|        feature_4|         feature_5|         feature_6|         feature_7|         feature_8|         feature_9|        feature_10|        feature_11|        feature_12| id|target|
+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+---+------+
|66.42466482734166| 44.70872653121246|30.775372350642026| 51.1389709391279| 28.72440341044677| 57.94952775008367| 51.28947730127982|11.669599638611906| 61.00306652060417|46.634124685340595|58.001695855553265| 31.69836310949832|  0|     0|
|70.87793616714605|  38.5992682438055|22.045

### Задание 2. Подготовить исходный набор данных для расчета критерия согласия Пирсона (Pearson’s Chi-squared test)

In [7]:
def row_to_vector(row):
  return [
    int(row[-1]),
    Vectors.dense(*row[:12])
  ]

df_prepared = df.rdd.map(row_to_vector).collect()
df_test = spark.createDataFrame(df_prepared, [ 'label', 'features' ])

df_test.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[66.4246648273416...|
|    0|[70.8779361671460...|
|    1|[52.6535876174208...|
|    1|[65.9166805541323...|
|    1|[65.6532396446104...|
+-----+--------------------+
only showing top 5 rows



### Задание 3.	Выполнить расчет критерия согласия Пирсона (Pearson’s Chi-squared test)

In [8]:
xi2 = ChiSquareTest.test(df_test, 'features', 'label', flatten=True) \
  .orderBy('featureIndex')

xi2.show()

+------------+-------------------+----------------+------------------+
|featureIndex|             pValue|degreesOfFreedom|         statistic|
+------------+-------------------+----------------+------------------+
|           0|0.48714614379548027|            1337|1338.0000000000587|
|           1|0.48714614379593424|            1337|1338.0000000000603|
|           2|0.48714614379548027|            1337|1338.0000000000587|
|           3| 0.4871461437959326|            1337|1338.0000000000605|
|           4|  0.487146143795963|            1337|1338.0000000000566|
|           5| 0.4871461437957113|            1337| 1338.000000000059|
|           6| 0.4871461437957113|            1337| 1338.000000000059|
|           7| 0.4871461437959539|            1337|1338.0000000000578|
|           8| 0.4871461437957102|            1337|1338.0000000000591|
|           9|0.48714614379548027|            1337|1338.0000000000587|
|          10| 0.4871461437957102|            1337|1338.0000000000591|
|     

### Задача 4. Записать полученные данные в файл (п. 3)

In [9]:
xi2.toPandas().to_json('data/r_4.json')